### Getting data from Kaggle using API

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d jangedoo/utkface-new

In [ ]:
!unzip '/content/utkface-new.zip'

### Preprocessing the data

In [ ]:
import os

len(os.listdir('/content/UTKFace'))

In [ ]:
os.listdir('/content/UTKFace')[100]

In [ ]:
age = []
gender = []
img_path =[]
for file in os.listdir('/content/UTKFace'):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [ ]:
len(age)

In [ ]:
gender[:5]

In [ ]:
age[:5]

In [ ]:
len(gender)

In [ ]:
import pandas as pd 

df = pd.DataFrame({'age':age, 'gender':gender, 'file':img_path})
df.head()

### Train-test Generator

In [ ]:
train = df.sample(frac=1, random_state=42)[:20000]

test = df.sample(frac=1, random_state=42)[20000:]

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip=True,
    vertical_flip= True
)

test_gen = ImageDataGenerator(
    rescale = 1./255
)

In [ ]:
train_generator = train_gen.flow_from_dataframe(
    train,
    directory='/content/UTKFace',
    x_col='file',  # Assuming 'file' column contains the paths to the images
    y_col=['age', 'gender'],  # 'age' and 'gender' are the output labels
    class_mode='multi_output'  # Specifying that we have multiple output labels
)

test_generator = test_gen.flow_from_dataframe(
    test,
    directory='/content/UTKFace',
    x_col='file',  # Assuming 'file' column contains the paths to the images
    y_col=['age', 'gender'],  # 'age' and 'gender' are the output labels
    class_mode='multi_output'  # Specifying that we have multiple output labels
)


In [ ]:
train_generator.class_mode

### Loading pre-trained resnet50 and changing the architecture according to required use

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
model = ResNet50(include_top = False, input_shape=(256,256,3))

In [ ]:
model.summary()

In [ ]:
model.trainable= False

In [ ]:
model.summary()

In [ ]:
output_resnet = model.layers[-1].output
output_resnet

In [ ]:
from keras.models import Model
from keras.layers import *

In [ ]:
flat = Flatten()(output_resnet)


hidden1 = Dense(512, activation='relu')(flat)
hidden2 = Dense(256, activation='relu')(flat)

hidden3 = Dense(256, activation='relu')(hidden1)
hidden4 = Dense(256, activation='relu')(hidden2)


output1 = Dense(1, activation='linear', name='age')(hidden3)
output2 = Dense(1,activation='sigmoid', name='gender')(hidden4)

In [ ]:
final_model = Model(inputs=model.input, outputs = [output1, output2])
# model.input

In [ ]:
final_model.summary()

In [ ]:
from keras.utils import plot_model

plot_model(final_model,show_shapes=True)

### Model Training

In [ ]:
final_model.compile(loss={'age':'mae', 'gender':'binary_crossentropy'}, optimizer='adam', metrics={'age':'mae', 'gender':'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
history = final_model.fit(train_generator, batch_size=32, validation_data=test_generator,epochs=10)

In [ ]:
history.history

In [ ]:
history.history['loss']

In [ ]:
history.history['accuracy']